In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt

# Carregar o dataset
data = pd.read_csv('Data/Google-Playstore.csv') #data é um DataFrame

# Limitar o dataset a 5000 linhas
#data = data.head(2000000)

# Inspecionar os primeiros registros e verificar valores ausentes
print("Dados iniciais:")
print(data.head())
print(data.info())
print("Valores ausentes antes da imputação:")
print(data.isnull().sum())

# Remover linhas onde a variável alvo 'Installs' é nula
data = data.dropna(subset=['Installs'])

# Converter a coluna 'Installs' para um formato numérico
data['Installs'] = data['Installs'].str.replace('[+,]', '', regex=True)
data['Installs'] = pd.to_numeric(data['Installs'], errors='coerce').astype('Int64')

# Verificar novamente alguns exemplos após a conversão
print("Exemplos de valores na coluna 'Installs' após a conversão:")
print(data['Installs'].head(10))

# Atualizar os valores ausentes após a remoção de linhas
print("Valores ausentes após remoção de linhas com 'Installs' nulos:")
print(data.isnull().sum())

# Preparação dos dados
# Selecionar apenas as variáveis especificadas
selected_features = ['Rating', 'Rating Count', 'Price', 'Size']
X = data[selected_features]
y = data['Installs']

# Verificar se ainda existem valores ausentes em y
print(f"Valores ausentes em y: {y.isnull().sum()}")

# Identificar variáveis numéricas e categóricas
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Preprocessamento
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputação de valores ausentes
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Imputação de valores ausentes
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Dividir o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Função para avaliar o modelo e retornar o pipeline treinado
def evaluate_model(name, model, X_train, y_train, X_test, y_test):
    print(f"Tipo de 'y' (Installs) antes do treino: {y_train.dtype}")

    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])

    pipeline.fit(X_train, y_train)
    X_train_transformed = pipeline.named_steps['preprocessor'].transform(X_train)
    X_test_transformed = pipeline.named_steps['preprocessor'].transform(X_test)
    print(f"Valores ausentes após imputação (treino) para o modelo {name}:")
    print(pd.DataFrame(X_train_transformed).isnull().sum())
    y_pred = pipeline.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f"Model: {name}")
    print(f"R^2: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}\n")
    return pipeline

# Imprimir tamanho do dataframe
print("Tamanho atual do dataframe:", data.shape)


Dados iniciais:
                                            App Name  \
0                                            Gakondo   
1                                Ampere Battery Info   
2                                             Vibook   
3  Smart City Trichy Public Service Vehicles 17UC...   
4                                            GROW.me   

                       App Id       Category  Rating  Rating Count Installs  \
0         com.ishakwe.gakondo      Adventure     0.0           0.0      10+   
1  com.webserveis.batteryinfo          Tools     4.4          64.0   5,000+   
2        com.doantiepvien.crm   Productivity     0.0           0.0      50+   
3     cst.stJoseph.ug17ucs548  Communication     5.0           5.0      10+   
4        com.horodyski.grower          Tools     0.0           0.0     100+   

   Minimum Installs  Maximum Installs  Free  Price  ...  \
0              10.0                15  True    0.0  ...   
1            5000.0              7662  True    0.0  ..

In [9]:
# Avaliar modelo e obter o pipeline treinado
pipeline = evaluate_model('Linear Regression', LinearRegression(), X_train, y_train, X_test, y_test)
#Imprimir tamanho do dataframe
print("Tamanho atual do dataframe:", data.shape)

Tipo de 'y' (Installs) antes do treino: Int64
Valores ausentes após imputação (treino) para o modelo Linear Regression:
0    0
dtype: int64
Model: Linear Regression
R^2: 0.4678
MAE: 226692.2062
MSE: 77983062246372.1719
RMSE: 8830801.9028

Tamanho atual do dataframe: (2312837, 24)


In [10]:
# Carregar o ficheiro CSV da nova app
new_app_df = pd.read_csv('new_app.csv')  # Substitua 'path/to/' pelo caminho correto do seu ficheiro CSV

# Remover colunas não presentes nos dados de treino
new_app_df = new_app_df[X.columns]

# Função para prever a variável alvo para uma nova app
def predict_new_app(pipeline, new_app):
    # Preprocessar a nova app
    new_app_transformed = pipeline.named_steps['preprocessor'].transform(new_app)
    # Prever a variável alvo
    prediction = pipeline.named_steps['model'].predict(new_app_transformed)
    return prediction

# Fazer a previsão utilizando o pipeline treinado anteriormente
prediction = predict_new_app(pipeline, new_app_df)
print(f"Previsão de 'Installs' para a nova app 'Gakondo': {prediction[0]:.0f}")
#Imprimir tamanho do dataframe
print("Tamanho atual do dataframe:", data.shape)

Previsão de 'Installs' para a nova app 'Gakondo': -29916
Tamanho atual do dataframe: (2312837, 24)
